In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
#training using 11 classes, but one hidden layer

mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

  # Create the model
x = tf.placeholder(tf.float32, [None, 784])
#W_in = tf.Variable(tf.zeros([784, 100]))
W_in = tf.Variable(tf.random_normal([784, 100], stddev=0.2))
W_h = tf.Variable(tf.random_normal([100, 11], stddev=0.2))
#W_h = tf.Variable(tf.zeros([100, 11]))
b = tf.Variable(tf.zeros([11]))

mult = tf.matmul(x, W_in) 
y = tf.matmul(mult, W_h) + b

#y = tf.nn.softmax(y) no activation function

rand = abs(np.random.normal(0,0.5,(2000,784)))
mnist_train = np.array(mnist.train.images[0:20000])
#mnist_train = np.array(mnist.train.images) #55000 entries
input_x = np.concatenate((mnist_train, rand), axis=0) #57000 entries #22000 entries

  # Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 11])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

label_mnist_last = np.zeros([20000,1])
labels_mnist = np.concatenate((mnist.train.labels[0:20000], label_mnist_last), axis=1)
labels_rnd = np.concatenate((np.zeros([2000,10]), np.ones([2000,1])), axis=1)
labels = np.concatenate((labels_mnist, labels_rnd), axis=0)

sess = tf.InteractiveSession()

tf.initialize_all_variables().run()
for _ in range(300):
    batch_xs = input_x
    batch_ys = labels
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [32]:
#calculating accuracy for test
rand_in_tst = abs(np.random.normal(0,0.5,(1000,784)))
mnist_test = np.array(mnist.test.images)
input_tst = np.concatenate((mnist_test, rand_in_tst), axis=0)

label_mnist_last_tst = np.zeros([10000,1])
labels_mnist_tst = np.concatenate((mnist.test.labels, label_mnist_last_tst), axis=1)
labels_rnd_tst = np.concatenate((np.zeros([1000,10]), np.ones([1000,1])), axis=1)
labels_tst = np.concatenate((labels_mnist_tst, labels_rnd_tst), axis=0)

weights_in = np.array(sess.run(W_in))
weights_hidden = np.array(sess.run(W_h))
biases = np.array(sess.run(b))

mult_test = np.dot(input_tst, weights_in)
outputs_test = np.dot(mult_test, weights_hidden) + biases

correct_prediction = tf.equal(tf.argmax(outputs_test, 1), tf.argmax(labels_tst, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test accuracy: %f" % sess.run(accuracy, feed_dict={x:input_tst,
                                      y_: labels_tst}))

Test accuracy: 0.919000
